### Assignment

Create a Dataset of movies which will be having three columns i.e.
+ Movie Name(Feature1)
+ Description(Feature2)
+ Genre(Target)

After Creating the dataset, apply NLP text Preprocessing Techniques to required columns.

### Solution

1. Get the Data from a Repo or url link

In [2]:
import json
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [130]:
movie_name = []
overview = []
genre = []

for i in range(1,100):
    response = requests.get(f'https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page={i}')
    movie_data_per_page = response.json()
    
    for i in movie_data_per_page['results']:
        movie_name.append(i['title'])
        overview.append(i['overview'])
        genre.append(i['genre_ids'])
        
frames = {'Movie':movie_name, 'Overview':overview, 'Genre':genre}
df = pd.DataFrame(frames)

print(df)

In [4]:
frames = {'Movie':movie_name, 'Overview':overview, 'Genre':genre}
df = pd.DataFrame(frames)
df

,Movie,Overview,Genre
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]"
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[18, 80]"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]"
4,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","[35, 18, 10749]"
5,12 Angry Men,The defense and the prosecution have rested an...,[18]
6,Spirited Away,"A young girl, Chihiro, becomes trapped in a st...","[16, 10751, 14]"
7,Parasite,"All unemployed, Ki-taek's family takes peculia...","[35, 53, 18]"
8,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[18, 28, 80, 53]"
9,The Green Mile,A supernatural tale set on death row in a Sout...,"[14, 18, 80]"


#### Reading the Genre IDs from another URL

In [8]:
response_genre = requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US')

response_genre_json = response_genre.json()

genre_dict = dict()

for i in response_genre_json['genres']:
    genre_dict[i['id']] = i['name']

print(genre_dict)

{28: 'Action', 12: 'Adventure', 16: 'Animation', 35: 'Comedy', 80: 'Crime', 99: 'Documentary', 18: 'Drama', 10751: 'Family', 14: 'Fantasy', 36: 'History', 27: 'Horror', 10402: 'Music', 9648: 'Mystery', 10749: 'Romance', 878: 'Science Fiction', 10770: 'TV Movie', 53: 'Thriller', 10752: 'War', 37: 'Western'}


In [10]:
# genre_dict[80]

In [11]:
df.dtypes

Movie       object
Overview    object
Genre       object
dtype: object

#### Converting the Genre IDs to the text

In [12]:
genre_values = df['Genre'].values
processedGenre = list()

for i in genre_values:
    temp = []
    for j in i:
        temp.append(genre_dict[j])
    processedGenre.append(temp)
    
    
df['GenreProcessed'] = processedGenre

In [14]:
df.head()

,Movie,Overview,Genre,GenreProcessed
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]","[Drama, Crime]"
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[18, 80]","[Drama, Crime]"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]","[Drama, Crime]"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]","[Drama, History, War]"
4,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","[35, 18, 10749]","[Comedy, Drama, Romance]"


2. PreProcess the text Data using NLP.

a. Lowercasing the text

In [20]:
def lowerCase(text):
    return text.lower()

processedTexts = []
inputTexts = df['Overview'].values
for i in inputTexts:
    processedTexts.append(lowerCase(i))

df['LowerCasedText'] = processedTexts

b. Removing unawanted tags, urls, links

In [24]:
import re

In [31]:
def removeTagsUrls(text):
    pattern = re.compile('^www.*com|^www.*in|<.*>')
    return pattern.sub(r'',text)


urlProcessedTexts = []
inputTexts = df['LowerCasedText'].values
for i in inputTexts:
    urlProcessedTexts.append(removeTagsUrls(i))
    
df['TagUrlRemovedText'] = urlProcessedTexts

c. Handle acronyms & chat word handling

In [49]:
f = open('sms_slang_translator.txt')
d = dict()
for i in f.readlines():
    arr = i.split('=')
    try:
        d[arr[0]] = arr[1]
    except:
        continue

In [52]:
def handleAcronyms(text):
    arr = text.split(' ')
    processedWords = []
    for i in arr:
        if i in d:
            processedWords.append(d[i])
        else:
            processedWords.append(i)
    return ' '.join(processedWords)


l = df['TagUrlRemovedText'].values
acronymProcessed = []
for i in l:
    acronymProcessed.append(handleAcronyms(i))
    
df['AcronymProcessed'] = acronymProcessed

In [53]:
df

,Movie,Overview,Genre,GenreProcessed,LowerCasedText,TagUrlRemovedText,AcronymProcessed
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]","[Drama, Crime]","spanning the years 1945 to 1955, a chronicle o...","spanning the years 1945 to 1955, a chronicle o...","spanning the years 1945 to 1955, a chronicle o..."
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[18, 80]","[Drama, Crime]",framed in the 1940s for the double murder of h...,framed in the 1940s for the double murder of h...,framed in the 1940s for the double murder of h...
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]","[Drama, Crime]",in the continuing saga of the corleone crime f...,in the continuing saga of the corleone crime f...,in the continuing saga of the corleone crime f...
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]","[Drama, History, War]",the true story of how businessman oskar schind...,the true story of how businessman oskar schind...,the true story of how businessman oskar schind...
4,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","[35, 18, 10749]","[Comedy, Drama, Romance]","raj is a rich, carefree, happy-go-lucky second...","raj is a rich, carefree, happy-go-lucky second...","raj is a rich, carefree, happy-go-lucky second..."
5,12 Angry Men,The defense and the prosecution have rested an...,[18],[Drama],the defense and the prosecution have rested an...,the defense and the prosecution have rested an...,the defense and the prosecution have rested an...
6,Spirited Away,"A young girl, Chihiro, becomes trapped in a st...","[16, 10751, 14]","[Animation, Family, Fantasy]","a young girl, chihiro, becomes trapped in a st...","a young girl, chihiro, becomes trapped in a st...","a young girl, chihiro, becomes trapped in a st..."
7,Parasite,"All unemployed, Ki-taek's family takes peculia...","[35, 53, 18]","[Comedy, Thriller, Drama]","all unemployed, ki-taek's family takes peculia...","all unemployed, ki-taek's family takes peculia...","all unemployed, ki-taek's family takes peculia..."
8,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[18, 28, 80, 53]","[Drama, Action, Crime, Thriller]",batman raises the stakes in his war on crime. ...,batman raises the stakes in his war on crime. ...,batman raises the stakes in his war on crime. ...
9,The Green Mile,A supernatural tale set on death row in a Sout...,"[14, 18, 80]","[Fantasy, Drama, Crime]",a supernatural tale set on death row in a sout...,a supernatural tale set on death row in a sout...,a supernatural tale set on death row in a sout...


d. Remove Punctuation

In [75]:
import string

punctuation = string.punctuation

# punctuation[22]

# s = string.punctuation
# s = s.replace(s[22],'')
# s = '[' + s + ']'
# pattern = re.compile(s)
# pattern.sub(r'','abc$wer')

'abcwer'

In [93]:
def removePunctuation(text):
    s = string.punctuation
    s = s.replace(s[22],'')
    pattern = re.compile('['+s+']')
    return pattern.sub(r'',text)

inputTexts = df['AcronymProcessed'].values
processedTextPunc = []
for i in inputTexts:
    processedTextPunc.append(removePunctuation(i))

df['PunctuationRemovedText'] = processedTextPunc

In [94]:
df

,Movie,Overview,Genre,GenreProcessed,LowerCasedText,TagUrlRemovedText,AcronymProcessed,PunctuationRemovedText
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]","[Drama, Crime]","spanning the years 1945 to 1955, a chronicle o...","spanning the years 1945 to 1955, a chronicle o...","spanning the years 1945 to 1955, a chronicle o...",spanning the years 1945 to 1955 a chronicle of...
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[18, 80]","[Drama, Crime]",framed in the 1940s for the double murder of h...,framed in the 1940s for the double murder of h...,framed in the 1940s for the double murder of h...,framed in the 1940s for the double murder of h...
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]","[Drama, Crime]",in the continuing saga of the corleone crime f...,in the continuing saga of the corleone crime f...,in the continuing saga of the corleone crime f...,in the continuing saga of the corleone crime f...
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]","[Drama, History, War]",the true story of how businessman oskar schind...,the true story of how businessman oskar schind...,the true story of how businessman oskar schind...,the true story of how businessman oskar schind...
4,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","[35, 18, 10749]","[Comedy, Drama, Romance]","raj is a rich, carefree, happy-go-lucky second...","raj is a rich, carefree, happy-go-lucky second...","raj is a rich, carefree, happy-go-lucky second...",raj is a rich carefree happygolucky second gen...
5,12 Angry Men,The defense and the prosecution have rested an...,[18],[Drama],the defense and the prosecution have rested an...,the defense and the prosecution have rested an...,the defense and the prosecution have rested an...,the defense and the prosecution have rested an...
6,Spirited Away,"A young girl, Chihiro, becomes trapped in a st...","[16, 10751, 14]","[Animation, Family, Fantasy]","a young girl, chihiro, becomes trapped in a st...","a young girl, chihiro, becomes trapped in a st...","a young girl, chihiro, becomes trapped in a st...",a young girl chihiro becomes trapped in a stra...
7,Parasite,"All unemployed, Ki-taek's family takes peculia...","[35, 53, 18]","[Comedy, Thriller, Drama]","all unemployed, ki-taek's family takes peculia...","all unemployed, ki-taek's family takes peculia...","all unemployed, ki-taek's family takes peculia...",all unemployed kitaeks family takes peculiar i...
8,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[18, 28, 80, 53]","[Drama, Action, Crime, Thriller]",batman raises the stakes in his war on crime. ...,batman raises the stakes in his war on crime. ...,batman raises the stakes in his war on crime. ...,batman raises the stakes in his war on crime w...
9,The Green Mile,A supernatural tale set on death row in a Sout...,"[14, 18, 80]","[Fantasy, Drama, Crime]",a supernatural tale set on death row in a sout...,a supernatural tale set on death row in a sout...,a supernatural tale set on death row in a sout...,a supernatural tale set on death row in a sout...


In [95]:
df['AcronymProcessed'][0]

'spanning the years 1945 to 1955, a chronicle of the fictional italian-american corleone crime family. when organized crime family patriarch, vito corleone barely survives an attempt on his life, his youngest son, michael steps in to take care of the would-be killers, launching a campaign of bloody revenge.'

In [96]:
df['PunctuationRemovedText'][0]

'spanning the years 1945 to 1955 a chronicle of the fictional italianamerican corleone crime family when organized crime family patriarch vito corleone barely survives an attempt on his life his youngest son michael steps in to take care of the wouldbe killers launching a campaign of bloody revenge'

In [98]:
print(len(df['AcronymProcessed'][0]))
print(len(df['PunctuationRemovedText'][0]))

307
298


e. Remove Stopwords from the text 

#### Find the list of stop words

In [115]:
f = open('stop_words.txt','r')
stopWords = []
for i in f.readlines():
    if i != '\n':
        stopWords.append(i[0:-1])

In [125]:
def removeStopWords(text):
    splitedWords = text.split(' ')
    processedWords = []
    for i in splitedWords:
        if i in stopWords:
            continue
        else:
            processedWords.append(i)
    return ' '.join(processedWords)

inputTexts = df['AcronymProcessed'].values
processedTextStopWords = []
for i in inputTexts:
    processedTextStopWords.append(removeStopWords(i))

df['RemovedStopWords'] = processedTextStopWords

In [129]:
(df['PunctuationRemovedText'][0])

'spanning the years 1945 to 1955 a chronicle of the fictional italianamerican corleone crime family when organized crime family patriarch vito corleone barely survives an attempt on his life his youngest son michael steps in to take care of the wouldbe killers launching a campaign of bloody revenge'

In [128]:
(df['RemovedStopWords'][0])

'spanning years 1945 1955, chronicle fictional italian-american corleone crime family. organized crime family patriarch, vito corleone barely survives attempt life, youngest son, michael steps take care would-be killers, launching campaign bloody revenge.'

f. Tokenisation

f. Stemming

g . Lemmatization